In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer

import matplotlib.pyplot as plt
from matplotlib import colors

In [2]:
#follows https://github.com/XanaduAI/all-you-need-is-spin
class Spin_2(qml.operation.Operation):
    num_wires = 2

    @staticmethod
    def compute_decomposition(theta, wires):
        schur2 = np.array([
            [1, 0, 0, 0],
            [0, 1/np.sqrt(2), 1/np.sqrt(2), 0],
            [0, 0, 0, 1],
            [0, 1/np.sqrt(2), -1/np.sqrt(2), 0]
        ])
    
        return [qml.QubitUnitary(schur2, wires = wires),
                qml.ControlledPhaseShift(theta, wires = wires),
                qml.adjoint(qml.QubitUnitary)(schur2, wires = wires)]

def create_singlet(i, j):
    qml.Hadamard(wires=i)
    qml.PauliZ(wires=i)
    qml.CNOT(wires=[i, j])
    qml.PauliX(wires=j)

In [3]:
def create_Heisenberg(N, J1, J2):
    H = sum([J1 * qml.PauliZ(i) @ qml.PauliZ((i + 1) % N) for i in range(N)])
    H += sum([J1 * qml.PauliX(i) @ qml.PauliX((i + 1) % N) for i in range(N)])
    H += sum([J1 * qml.PauliY(i) @ qml.PauliY((i + 1) % N) for i in range(N)])

    H += sum([J2 * qml.PauliZ(i) @ qml.PauliZ((i + 2) % N) for i in range(N)])
    H += sum([J2 * qml.PauliX(i) @ qml.PauliX((i + 2) % N) for i in range(N)])
    H += sum([J2 * qml.PauliY(i) @ qml.PauliY((i + 2) % N) for i in range(N)])

    return H

def prepare_init_state(N):
    for i in range(0, N, 2):
        create_singlet(i, i+1)

In [4]:
dev = qml.device("default.qubit", wires=4)
N = 4

ham = create_Heisenberg(N, 1.0, 0.44)
# ham_sparse = qml.SparseHamiltonian(ham.sparse_matrix(), wires=range(N))

@qml.qnode(dev)
def circuit_2qubits(params):
    prepare_init_state(N)
    
#     p0 = [0.5,0.3,0.2]
#     qml.Rot(*p0, wires=0)
#     qml.Rot(*p0, wires=1)
#     qml.Rot(*p0, wires=2)
#     qml.Rot(*p0, wires=3)
    
    k = 0
    for l in range(4):
        for i in range(0, N, 2):
            Spin_2(params[k], wires=[i, (i + 1) % N])
            k += 1

        for i in range(1, N, 2):
            Spin_2(params[k], wires=[i, (i + 1) % N])
            k += 1

        for i in range(0, N):
            Spin_2(params[k], wires=[i, (i + 2) % N])
            k += 1

    return qml.expval(ham)

In [5]:
#testing to see if the circuits is indeed SU(2) invariant under SU(2) rotation
Params = pnp.random.rand(50, requires_grad=False)

K = circuit_2qubits(Params)

print(K)

-5.841205381008374


In [6]:
# run this cell afer induce an overall rotation in the circuit
@qml.qnode(dev)
def circuit_2qubits(params):
    prepare_init_state(N)
    
    #some arbitrary parameters
    p0 = [0.5,0.3,0.2]
    qml.Rot(*p0, wires=0)
    qml.Rot(*p0, wires=1)
    qml.Rot(*p0, wires=2)
    qml.Rot(*p0, wires=3)
    
    k = 0
    for l in range(4):
        for i in range(0, N, 2):
            Spin_2(params[k], wires=[i, (i + 1) % N])
            k += 1

        for i in range(1, N, 2):
            Spin_2(params[k], wires=[i, (i + 1) % N])
            k += 1

        for i in range(0, N):
            Spin_2(params[k], wires=[i, (i + 2) % N])
            k += 1

    return qml.expval(ham)

K = circuit_2qubits(Params)

print(K)

-5.841205381008365


In [7]:
# run this cell afer induce a rotation on only one qubit in the circuit should change the result
@qml.qnode(dev)
def circuit_2qubits(params):
    prepare_init_state(N)
    
    p0 = [0.5,0.3,0.2]
    qml.Rot(*p0, wires=0)
#     qml.Rot(*p0, wires=1)
#     qml.Rot(*p0, wires=2)
#     qml.Rot(*p0, wires=3)
    
    k = 0
    for l in range(4):
        for i in range(0, N, 2):
            Spin_2(params[k], wires=[i, (i + 1) % N])
            k += 1

        for i in range(1, N, 2):
            Spin_2(params[k], wires=[i, (i + 1) % N])
            k += 1

        for i in range(0, N):
            Spin_2(params[k], wires=[i, (i + 2) % N])
            k += 1

    return qml.expval(ham)

K = circuit_2qubits(Params)

print(K)

-5.329856917829823


In [9]:
# run this cell afer induce a rotation on two of the neighbouring qubit in the circuit does not change the result, which is by design but unwanted.
@qml.qnode(dev)
def circuit_2qubits(params):
    prepare_init_state(N)
    
    p0 = [0.5,0.3,0.2]
    qml.Rot(*p0, wires=0)
    qml.Rot(*p0, wires=1)
#     qml.Rot(*p0, wires=2)
#     qml.Rot(*p0, wires=3)
    
    k = 0
    for l in range(4):
        for i in range(0, N, 2):
            Spin_2(params[k], wires=[i, (i + 1) % N])
            k += 1

        for i in range(1, N, 2):
            Spin_2(params[k], wires=[i, (i + 1) % N])
            k += 1

        for i in range(0, N):
            Spin_2(params[k], wires=[i, (i + 2) % N])
            k += 1

    return qml.expval(ham)

K = circuit_2qubits(Params)

print(K)

-5.841205381008367


In [11]:
# run this cell afer induce a rotation on two of the non-neighbouring qubit in the circuit does change the result.
@qml.qnode(dev)
def circuit_2qubits(params):
    prepare_init_state(N)
    
    p0 = [0.5,0.3,0.2]
    qml.Rot(*p0, wires=0)
#     qml.Rot(*p0, wires=1)
    qml.Rot(*p0, wires=2)
#     qml.Rot(*p0, wires=3)
    
    k = 0
    for l in range(4):
        for i in range(0, N, 2):
            Spin_2(params[k], wires=[i, (i + 1) % N])
            k += 1

        for i in range(1, N, 2):
            Spin_2(params[k], wires=[i, (i + 1) % N])
            k += 1

        for i in range(0, N):
            Spin_2(params[k], wires=[i, (i + 2) % N])
            k += 1

    return qml.expval(ham)

K = circuit_2qubits(Params)

print(K)

-4.737422573249601
